In [548]:
import json

In [549]:
f = open('model2.json')

model = json.load(f)

f.close()

model

{'meta': {'name': 'Cloud Description',
  'description': 'Описание признаков облаков',
  'schemaVersion': '2'},
 'schema': [{'param': 'shining',
   'title': 'Солнце и Луна просвечивают',
   'description': 'Видно ли Солнце или Луну сквозь облака',
   'spec': {'type': 'bool'},
   'require': True},
  {'param': 'height',
   'title': 'Высота облаков',
   'description': 'Оценочная высота облаков в метрах',
   'spec': {'type': 'enum',
    'data': [{'title': 'Очень высоко',
      'description': 'Облака находятся очень высоко над землей',
      'image': 'sample.jpg',
      'value': 'very_high'},
     {'title': 'Высоко',
      'description': 'Облака находятся высоко над землей',
      'image': 'sample.jpg',
      'value': 'high'},
     {'title': 'Средне',
      'description': 'Облака находятся средне над землей, ни высоко, ни низко',
      'image': 'sample.jpg',
      'value': 'middle'},
     {'title': 'Низко',
      'description': 'Облака находятся низко над землей',
      'image': 'sample.jpg',

In [550]:
f = open('data2.json')

data = json.load(f)

f.close()

data

{'meta': {'name': 'Cloud Classificator',
  'description': 'Классификатор облаков на основании признаков',
  'schemaVersion': '2'},
 'stack': [{'id': 'type',
   'title': 'Тип облаков',
   'description': 'Тип облака определяется условием образования',
   'limit': 1},
  {'id': 'morphology',
   'title': 'Морфологическая классификация',
   'description': 'На основании структуры и внешнего вида облака'},
  {'id': 'attention',
   'title': 'Предупреждение о опасностях',
   'description': 'Возможные опасные погодные явления, сопутствующие данному виду облаков'}],
 'graph': [{'type': 'condition',
   'comment': 'Это просвечивающиеся облака или нет?',
   'statements': [{'type': 'statement',
     'param': 'shining',
     'predicate': '=',
     'value': True,
     'require': True,
     'score': 1}],
   'graph': [{'type': 'condition',
     'comment': 'Высоко ли облака?',
     'statements': [{'type': 'statement',
       'param': 'height',
       'predicate': '=',
       'value': 'middle',
       'requ

In [551]:
obj = {}

for param in model['schema']:
    obj[param['param']] = None

obj

{'shining': None,
 'height': None,
 'visual_morph': None,
 'color': None,
 'season': None,
 'speed': None}

In [552]:
obj['height'] = 'middle'
# obj['ground_temperature'] = 50
obj['color'] = 'gray'
obj['shining'] = False
obj

{'shining': False,
 'height': 'middle',
 'visual_morph': None,
 'color': 'gray',
 'season': None,
 'speed': None}

In [553]:
stack = {}

def stack_flush():
    for s in data['stack']:
        stack[s['id']] = []

def stack_result(id, result, score):
    result['score'] = score
    stack[id].append(result)

In [554]:
class Node():
    def __init__(self, graph, depth = 0):
        self.type = graph['type']
        self.graph = graph
        self.nodes = []
        self.depth = depth
        self.score = 0
        
        try:
            self.stack = graph['stack']
        except:
            self.stack = None
        
        try:
            self.comment = graph['comment']
        except:
            self.comment = None

        try:
            for g in graph['graph']:
                self.nodes.append(Node(g, depth + 1))
        except:
            pass


        match self.type:
            case 'condition':
                self.statements = graph['statements']
            case 'report':
                self.report = graph['report']

    def debug_trace(self, trace = []):
        trace.append(self)
        for node in self.nodes:
            node.debug_trace(trace)

        return trace
    
    def debug_info(self):
        return {
            'node_count': len(self.nodes), 
            'depth': self.depth,
            'type': self.type
        }
    
    def debug_tree(self):
        for n in self.debug_trace():
            info = n.debug_info()
            print('     ' * n.depth + str(info))

    def execute_condition(self):
        true_count = 0
        global_score = 0

        for statement in self.statements:
            try:
                st_score = statement['score']
            except:
                st_score = 0


            if statement['param'] in obj:
                if obj[statement['param']] is not None:
                    values = []
                    if type(statement['value']) is not list:
                        values.append(statement['value'])
                    else:
                        values = statement['value']
                    
                    for dst in values:
                        ost = obj[statement['param']]

                        match statement['predicate']:
                            case '>':
                                if float(ost) > float(dst):
                                    true_count += 1
                                    global_score += st_score
                            case '<':
                                if float(ost) < float(dst):
                                    true_count += 1
                                    global_score += st_score
                            case '=':
                                if ost == dst:
                                    true_count += 1
                                    global_score += st_score
                            case '>=':
                                if float(ost) >= float(dst):
                                    true_count += 1
                                    global_score += st_score
                            case '<=':
                                if float(ost) <= float(dst):
                                    true_count += 1
                                    global_score += st_score
                            case '!=':
                                if float(ost) != float(dst):
                                    true_count += 1
                                    global_score += st_score
                            case _:
                                pass
                else:
                    if not statement['require']:
                        true_count += 1
        
        if true_count > 0:
            self.execute_nodes(global_score + self.score)

    def execute(self, score = 0):
        self.score = score

        match self.type:
            case 'report':
                if 'requireScore' in self.report:
                    if self.report['requireScore'] <= self.score:
                        stack_result(self.stack, self.report, self.score)
                        self.execute_nodes(self.score)
                else:
                    stack_result(self.stack, self.report, self.score)
                    self.execute_nodes(self.score)
            case 'condition':
                self.execute_condition()

            # case _:
            #     self.execute_nodes(self.score)
    
    def execute_nodes(self, score):
        for n in self.nodes:
            n.execute(score)


In [555]:
stack_flush()

for i in data['graph']:
    n = Node(i)
    n.debug_tree()
    n.execute()

print(json.dumps(stack, indent=4, ensure_ascii=False))

{'node_count': 2, 'depth': 0, 'type': 'condition'}
     {'node_count': 2, 'depth': 1, 'type': 'condition'}
          {'node_count': 0, 'depth': 2, 'type': 'report'}
          {'node_count': 0, 'depth': 2, 'type': 'report'}
     {'node_count': 2, 'depth': 1, 'type': 'condition'}
          {'node_count': 2, 'depth': 2, 'type': 'condition'}
               {'node_count': 0, 'depth': 3, 'type': 'report'}
               {'node_count': 0, 'depth': 3, 'type': 'report'}
          {'node_count': 3, 'depth': 2, 'type': 'condition'}
               {'node_count': 1, 'depth': 3, 'type': 'condition'}
                    {'node_count': 0, 'depth': 4, 'type': 'report'}
               {'node_count': 1, 'depth': 3, 'type': 'condition'}
                    {'node_count': 0, 'depth': 4, 'type': 'report'}
               {'node_count': 1, 'depth': 3, 'type': 'condition'}
                    {'node_count': 0, 'depth': 4, 'type': 'report'}
{'node_count': 2, 'depth': 0, 'type': 'condition'}
     {'node_count': 

In [556]:
stack_flush()
n.execute()

print(json.dumps(stack, indent=4, ensure_ascii=False))

{
    "type": [
        {
            "title": "Слоисто-кучевые облака (Stratocumulus, Sc)",
            "description": "Слоисто-кучевые облака (лат. Stratocumulus, Sc) — крупные серые гряды пластин или хлопьев, разделённые просветами, либо сливающиеся в сплошной покров. Образуются на высоте 0,6—1,5 км. Состоят в основном из мелких капелек воды радиусом 5—7 мкм с колебаниями от 1 до 60 мкм, зимой переохлаждённых. Из большинства разновидностей слоисто-кучевых облаков осадки, как правило, не выпадают. Из непросвечивающих слоисто-кучевых (Sc op.) может выпасть слабый дождь или редкий снег.",
            "requireScore": 1,
            "score": 3
        }
    ],
    "morphology": [],
    "attention": [
        {
            "title": "Возможны сильные осадки!",
            "requireScore": 1,
            "score": 3
        }
    ]
}
